In [1]:
# module imports (B)

from fpds import get_fpds_search_results
from fpds import write_fpds_results_to_file
from fpds import extract_fpds_fields_to_list_of_tuples

import pyodbc
import pandas as pd

In [ ]:
# possible test cases

def getCDC5yr():
    '''my data extract to see if I can make the webinar data extract and preparation process repeatable'''
    data = {}
    data['initial_search_url'] = 'https://www.fpds.gov/ezsearch/FEEDS/ATOM?FEEDNAME=PUBLIC&q=AGENCY_CODE:7523+LAST_MOD_DATE:[2018/10/01,2024/09/30]&templateName=1.5.3&indexName=awardfull&x=24&y=11&sortBy=LAST_MOD_DATE&asc=Y'
    data['output_file'] = r'C:\Users\johnp\OneDrive - Delphi Consulting Solutions\Database Projects\Project Independence\fpds\z_real_thing_07_13\all_cdc_0713.json'
    return data

def getNIH5yr():
    '''Brian request for NIH download for future webinar'''
    data = {}
    data['initial_search_url'] = 'https://www.fpds.gov/ezsearch/FEEDS/ATOM?FEEDNAME=PUBLIC&q=AGENCY_CODE:7529+LAST_MOD_DATE:[2018/10/01,2024/09/30]&templateName=1.5.3&indexName=awardfull&x=24&y=11&sortBy=LAST_MOD_DATE&asc=Y'
    data['output_file'] = r'C:\Users\johnp\OneDrive - Delphi Consulting Solutions\Database Projects\Project Independence\fpds\z_real_thing_07_13\NIH_5_yr.json'
    return data

def getTestData():
    '''testing file worth 1 day of updates (19 entries) for testing out my processess'''
    data = {}
    data['initial_search_url'] = 'https://www.fpds.gov/ezsearch/FEEDS/ATOM?FEEDNAME=PUBLIC&q=AGENCY_CODE:7523+CREATED_DATE:[2021/10/01,2021/10/01]&templateName=1.5.3&indexName=awardfull&x=24&y=11&sortBy=CREATED_DATE&asc=Y'
    data['output_file'] = r'C:\Users\johnp\OneDrive - Delphi Consulting Solutions\Delphics\Project Independence\API Downloads\TESTING_0714.json'
    return data

In [ ]:
# get inputs

# input = getTestData()
input = getCDC5yr()

initial_search_url = input['initial_search_url']
output_file = input['output_file']

In [ ]:
# run test case to extract data from api and split results into entries and metadata

rs = get_fpds_search_results(initial_search_url, return_json=False)
entries = rs['entries']
metadata = rs['metadata']

# print(json.dumps(entries, indent=4))
# print('\n\n\n')
# print(json.dumps(metadata, indent=4))

In [ ]:
# write extracted entries data to file (JSON file)
write_fpds_results_to_file(entries, output_file)

In [2]:
# read entries from JSON file (B)



if 1==0:
    # use the output_file from the original code
    pass
else:
    # Use this block if you want to use CDC5yr file without running the extract process again (45 mins)
    # output_file = r'C:\Users\johnp\OneDrive - Delphi Consulting Solutions\Delphics\Project Independence\API Downloads\all_cdc_0713_orig.json'
    output_file = r'C:\Users\johnp\OneDrive - Delphi Consulting Solutions\Delphics\Project Independence\API Downloads\CDC5yr.json'
    
    
f = pd.read_json(output_file)




In [3]:
# convert the entries to dict format from json format (B)
entries = f.to_dict()

# for keys,values in entries.items():
#     print(keys, values, '\n\n')



In [4]:
# extract fields and field values to list of tuples (B)

flds = extract_fpds_fields_to_list_of_tuples(entries)
# print(flds)

In [5]:
# list all field names (B)
field_names = []

for f in flds:
    if f[1] not in field_names:
        field_names.append(f[1])

for i in field_names:
    print(i)

awardAgencyID
awardModNumber
awardPIID
awardTransactionNumber
vendorIsCorporateEntityNotTaxExempt
vendorIsCorporateEntityTaxExempt
vendorIsInternationalOrganization
vendorIsPartnershipOrLimitedLiabilityPartnership
vendorIsSmallAgriculturalCooperative
vendorIsSolePropreitorship
vendorIsUSGovernmentEntity
competitionA76Action
competitionCommercialItemAcquisitionProcedures
competitionCommercialItemTestProgram
competitionExtentCompeted
competitionFedBizOpps
competitionNumberOfOffersReceived
competitionPriceEvaluationPercentDifference
competitionSmallBusinessCompetitivenessDemonstrationProgram
competitionTypeOfSetAside
contractConsolidatedContract
contractActionType
contractCostAccountingStandardsClause
contractGFEorGFP
contractMultiYearContract
contractNumberOfActions
contractPerformanceBasedServiceContract
contractPurchaseCardAsPaymentMethod
contractTypeOfContractPricing
contractUndefinitizedAction
marketingFeePaidForUseOfService
contractBaseAndAllOptionsValue
contractBaseAndExercisedOpti

In [ ]:
# connect to database

cn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                      "Server=delphics;"
                      "Database=independence;"
                      "Trusted_Connection=yes;")

# cn = pyodbc.connect(    "Driver={ODBC Driver 17 for SQL Server};"
#                         "Server=72.52.255.10\QHV6EG;"
#                         "Database=webinars;"
#                         "User Id=js"
#                         "Password=Purvis1477!;"
#                       )


cursor = cn.cursor()

# cn.close()

In [ ]:
# create imp.Stage01 table for storing results
sql =   '''
				if OBJECT_ID('imp.Stage01','u') is not null drop table imp.Stage01
                create table imp.Stage01 (entryNumber   int)
        '''

try:
    cursor.execute(sql)

except Exception as e:
    print(sql, '\n')
    print(e)

else:
    cursor.commit()

In [ ]:
# add all fields from field list to the table structure

exceptions = []

for f in field_names:
    sql = 'alter table imp.Stage01 add ' + f + ' varchar(5000)'
    try:
        cursor.execute(sql) 
    except Exception as e:
        exceptions.append((sql,e))
    else:
        cursor.commit()

if exceptions:
    print(exceptions)

In [ ]:
# OPTION 1 - PART A - populate entry numbers from metadata

# entry_count = metadata['entry_count']

# for i in range(entry_count):
#     sql = 'insert imp.Stage01 (entryNumber) values (' + str(i+1) + ') ' 
#     cursor.execute(sql)
#     cursor.commit()


In [ ]:
# OPTION 1 - PART B - populate individual fields from entries data in the flds variable

# exceptions_list = []
# sql_list = []

# for f in flds:
#     entryNumber    = f[0]
#     fieldName      = f[1] 
#     fieldValue     = f[2] 

#     try:
#         sql = f'''update imp.Stage01 set {fieldName} = '{fieldValue.replace("'","''") }' where entryNumber = {entryNumber}'''
#         cursor.execute(sql)

#     except Exception as e:
#         exceptions_list.append(e)
#         sql_list.append(sql)
#         print(sql, '\n')
#         print(e)

#     else:
#         cursor.commit()

# cn.close()

In [7]:
# OPTION 2 - create individual insert sql statements for each entry and write to a list (before writing to a file) (B)

sql_scripts = []
for entry_counter, entry in entries.items():
    flds = [(fld, entry[fld]) for fld in entry]

    fld = 'insert imp.Stage01 (entryNumber '
    val = 'values (' + str(entry_counter) + ' '
    for f in flds:
        fld += f", {f[0]} " 
        cleaned_value = str(f[1]).replace("'","''")
        val += f", '{cleaned_value}' "

    sql =  fld + ')' + '\n' + ' ' + val + ' )' + '\n'
    sql_scripts.append(sql)

    # print(sql)
    # print('\n')


In [ ]:
# ON HOLD







# DID NOT FINISH THIS SECTION 

# print(sql_scripts[-1])
# print(len(sql_scripts))

# cntr = 1

################ code below is NOT WORKING YET  for looping through sql_scripts ################
# batch_size = 10000
# sections = [(i, i+batch_size-1) for i in range(0,len(sql_scripts)-1,batch_size)]
# for s in sections: 
#     print(s)
#     batch = sql_scripts[s[0]:s[1]]
#     break

#     # print(  batch[0], '\n', batch[len(batch)])
#     # print('--------------------------------') 


# # for s in range(len(sql_scripts)):
# #     print(s)
# #     # data = sql_scripts[sections[s]::,batch_size]



# if 1==0:
#     for s in sections:
#         print(f"{s[0]} to {s[1]} ", '\n', sql_scripts[s[0]], '\n', sql_scripts[s[1]], '\n')
#         print('--------------------------------')
################ code below is NOT WORKING YET  for looping through sql_scripts ################


#for i in range(0,len(sql_scripts),batch_size):
    # print(f"{i} to {i+batch_size-1} ", '\n', sql_scripts[i], '\n', sql_scripts[i+batch_size-1], '\n')
    # print(f"{i} to {i+batch_size-1} ", '\n', sql_scripts[i], '\n')
#    print(f"{i} to {i+batch_size-1} ")
    # print('--------------------------------')
#    cntr+=1


In [ ]:
# ON HOLD


# for i in range(1,len(sql_scripts),1000):
#     file_cntr = '0000' + str(cntr)
#     file_name = 'cdc_5_yr_' + file_cntr + '.sql'
#     file = open(file_name, 'x')
#     for stmt in sql_scripts:
#         file.write()

#     file.write(sql)
#     file.write('\n')
#     cntr +=1

# file.close()
# print('file created successfully')


# f = open("cdc_5_yr.sql", "x")
# f.write(sql_script)
# f.close()

In [26]:
# THIS SCRIPT WORKS to CREATE MULTIPLE SQL STATEMENT FILES FROM THE DATA

# print(len(sql_scripts))

cntr = 1
batchsize = 5000
for i in range(0,65000,batchsize):
    file_counter = '0' + str(cntr)[0:2]
    file_name = 'cdc_5yr_' + file_counter + '.sql'
    file = open(file_name, 'w')
    start_rec = i
    stop_rec = i+batchsize
    batch = sql_scripts[start_rec:stop_rec]
    for rec in batch:
        file.write(rec + '\n')
    cntr +=1
    

    # print(start_rec,stop_rec)
    
    
 #   print(file_name)

# file_1 = open("cdc_5_yr_01.sql", "w")     - done
# for rec in sql_scripts[0:5000]:               
#     file_1.write(rec + '\n\n')
# file_1.close()

file.close()

